In [1]:
# Install required packages
!pip install gradio whisper-openai speechrecognition pydub librosa plotly matplotlib pandas numpy vosk

# Download Vosk model
!wget https://alphacephei.com/vosk/models/vosk-model-small-en-us-0.15.zip
!unzip -q vosk-model-small-en-us-0.15.zip

import gradio as gr
import whisper
import speech_recognition as sr
import librosa
import librosa.display
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import io
import tempfile
import os
from datetime import datetime
from pydub import AudioSegment
import wave
import json
from vosk import Model, KaldiRecognizer
from typing import Dict, List, Tuple
import time
import base64

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 52.4 MB/s eta 0:00:00
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22427 sha256=bbb2cc6bbe49feb1d341d4ea1293138ce0668958d40b75a988e87c8ad5d1706f
  Stored in directory: /root/.cache/pip/wheels/7e/75/5b/e1d5c3756631e4bda806f6cc9640153b39484bb6f7b0b8def3
Successfully built srt
--2025-10-01 08:38:37--  https://alphacephei.com/vosk/models/vosk-model-small-en-us-0.15.zip
Resolving alphacephei.com (alphacephei.com)... 188.40.21.16, 2a01:4f8:13a:279f::2
Connecting to alphacephei.com (alphacephei.com)|188.40.21.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41205931 (39M) [application/zip]
Saving to: ‘vosk-model-small-en-us-0.15.zip’

vosk-model-small-en 100%[===================>]  39.30M  17.4MB/

In [7]:
class STTEngine:
    def __init__(self):
        self.recognizer = sr.Recognizer()
        self.results_history = []
        self.performance_metrics = []

    def process_audio(self, audio_path: str, method: str) -> Dict:
        """Process audio with specified method"""
        try:
            if method == "Whisper":
                return self._recognize_whisper(audio_path)
            elif method == "Vosk":
                return self._recognize_vosk(audio_path)
            elif method == "Google API":
                return self._recognize_google(audio_path)
            else:
                return {
                    "text": "Invalid method selected",
                    "success": False,
                    "processing_time": 0,
                    "method": method
                }
        except Exception as e:
            return {
                "text": f"Error in {method}: {str(e)}",
                "success": False,
                "processing_time": 0,
                "method": method
            }

    def _recognize_whisper(self, audio_path: str) -> Dict:
        """Whisper recognition"""
        start_time = time.time()
        try:
            model = whisper.load_model("base")
            result = model.transcribe(audio_path)
            processing_time = time.time() - start_time
            return {
                "text": result["text"].strip(),
                "success": True,
                "processing_time": processing_time,
                "method": "Whisper"
            }
        except Exception as e:
            return {
                "text": f"Whisper Error: {str(e)}",
                "success": False,
                "processing_time": time.time() - start_time,
                "method": "Whisper"
            }

    def _recognize_vosk(self, audio_path: str) -> Dict:
        """Vosk recognition"""
        start_time = time.time()
        try:
            model = Model("vosk-model-small-en-us-0.15")

            # Ensure audio is in correct format
            if not audio_path.endswith('.wav'):
                audio_path = self._convert_to_wav(audio_path)

            wf = wave.open(audio_path, "rb")

            # Check audio format
            if wf.getnchannels() != 1 or wf.getsampwidth() != 2 or wf.getcomptype() != "NONE":
                converted_path = self._convert_audio_format(audio_path)
                wf.close()
                wf = wave.open(converted_path, "rb")

            rec = KaldiRecognizer(model, wf.getframerate())
            rec.SetWords(True)

            results = []
            while True:
                data = wf.readframes(4000)
                if len(data) == 0:
                    break
                if rec.AcceptWaveform(data):
                    part_result = json.loads(rec.Result())
                    results.append(part_result.get('text', ''))

            final_result = json.loads(rec.FinalResult())
            text = ' '.join(results + [final_result.get('text', '')]).strip()

            wf.close()
            return {
                "text": text if text else "No speech detected",
                "success": bool(text and text != "No speech detected"),
                "processing_time": time.time() - start_time,
                "method": "Vosk"
            }

        except Exception as e:
            return {
                "text": f"Vosk Error: {str(e)}",
                "success": False,
                "processing_time": time.time() - start_time,
                "method": "Vosk"
            }

    def _recognize_google(self, audio_path: str) -> Dict:
        """Google Speech API recognition"""
        start_time = time.time()
        try:
            # Convert to WAV if needed
            if not audio_path.endswith('.wav'):
                audio_path = self._convert_to_wav(audio_path)

            with sr.AudioFile(audio_path) as source:
                audio_data = self.recognizer.record(source)
                text = self.recognizer.recognize_google(audio_data)
                return {
                    "text": text,
                    "success": True,
                    "processing_time": time.time() - start_time,
                    "method": "Google API"
                }
        except sr.UnknownValueError:
            return {
                "text": "Could not understand audio",
                "success": False,
                "processing_time": time.time() - start_time,
                "method": "Google API"
            }
        except sr.RequestError as e:
            return {
                "text": f"Google API Error: {e}",
                "success": False,
                "processing_time": time.time() - start_time,
                "method": "Google API"
            }
        except Exception as e:
            return {
                "text": f"Google API Exception: {str(e)}",
                "success": False,
                "processing_time": time.time() - start_time,
                "method": "Google API"
            }

    def _convert_to_wav(self, audio_path: str) -> str:
        """Convert audio file to WAV format"""
        try:
            if audio_path.endswith('.wav'):
                return audio_path

            output_path = audio_path + '.wav'
            audio = AudioSegment.from_file(audio_path)
            audio.export(output_path, format="wav")
            return output_path
        except Exception as e:
            print(f"Conversion error: {e}")
            return audio_path

    def _convert_audio_format(self, audio_path: str) -> str:
        """Convert audio to proper format for Vosk"""
        try:
            output_path = "converted_audio.wav"
            audio = AudioSegment.from_file(audio_path)
            audio = audio.set_channels(1)  # Mono
            audio = audio.set_frame_rate(16000)  # 16kHz
            audio = audio.set_sample_width(2)  # 16-bit
            audio.export(output_path, format="wav")
            return output_path
        except Exception as e:
            print(f"Audio conversion error: {e}")
            return audio_path

    def compare_all_methods(self, audio_path: str, audio_type: str = "Custom") -> Dict:
        """Compare all three methods with proper error handling"""
        methods = ["Whisper", "Vosk", "Google API"]
        results = {}

        for method in methods:
            print(f"Processing with {method}...")
            result = self.process_audio(audio_path, method)
            # Use consistent keys (lowercase with underscore)
            key = method.lower().replace(' ', '_')
            results[key] = result
            self.performance_metrics.append(result)

        comparison_result = {
            'audio_type': audio_type,
            'timestamp': datetime.now(),
            **results
        }
        self.results_history.append(comparison_result)

        print(f"Comparison completed. Results keys: {list(comparison_result.keys())}")
        return comparison_result

# Initialize engine
stt_engine = STTEngine()

In [11]:
def create_performance_dashboard() -> go.Figure:
    """Create performance dashboard from history - FIXED VERSION"""
    if not stt_engine.performance_metrics:
        # Return empty figure if no data
        fig = go.Figure()
        fig.add_annotation(
            text="📊 No performance data available yet<br>Process some audio files to see analytics!",
            xref="paper", yref="paper",
            x=0.5, y=0.5,
            showarrow=False,
            font=dict(size=16, color="gray"),
            align="center"
        )
        fig.update_layout(
            title_text="Performance Analytics Dashboard",
            height=400
        )
        return fig

    # Create DataFrame from performance metrics
    df = pd.DataFrame(stt_engine.performance_metrics)

    # Debug: Print what we have
    print(f"Performance metrics columns: {df.columns.tolist()}")
    print(f"Methods found: {df['method'].unique()}")
    print(f"Total records: {len(df)}")

    # Calculate success rates
    success_rates = df.groupby('method')['success'].mean() * 100

    # Calculate average processing times
    avg_times = df.groupby('method')['processing_time'].mean()

    # Create dashboard with 2x2 layout
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            '🎯 Success Rates by Method',
            '⏱️ Average Processing Time',
            '📊 Processing Time Distribution',
            '📈 Performance Overview'
        ),
        specs=[
            [{"type": "bar"}, {"type": "bar"}],
            [{"type": "box"}, {"type": "scatter"}]
        ],
        vertical_spacing=0.12,
        horizontal_spacing=0.1
    )

    # 1. Success rates (Bar chart)
    if not success_rates.empty:
        fig.add_trace(
            go.Bar(
                x=success_rates.index,
                y=success_rates.values,
                name="Success Rate",
                marker_color=['#28a745' if rate > 50 else '#dc3545' for rate in success_rates.values],
                text=[f"{rate:.1f}%" for rate in success_rates.values],
                textposition='auto'
            ),
            row=1, col=1
        )

    # 2. Average processing times (Bar chart)
    if not avg_times.empty:
        fig.add_trace(
            go.Bar(
                x=avg_times.index,
                y=avg_times.values,
                name="Avg Processing Time",
                marker_color='#007bff',
                text=[f"{time:.2f}s" for time in avg_times.values],
                textposition='auto'
            ),
            row=1, col=2
        )

    # 3. Processing time distribution (Box plot)
    if not df.empty:
        for method in df['method'].unique():
            method_data = df[df['method'] == method]['processing_time']
            if len(method_data) > 0:
                fig.add_trace(
                    go.Box(
                        y=method_data,
                        name=method,
                        marker_color='#ffc107',
                        boxpoints='all',
                        jitter=0.3,
                        pointpos=-1.8
                    ),
                    row=2, col=1
                )

    # 4. Performance scatter plot (Success vs Processing Time)
    if not df.empty:
        color_map = {'Whisper': '#28a745', 'Vosk': '#007bff', 'Google API': '#dc3545'}

        for method in df['method'].unique():
            method_data = df[df['method'] == method]
            if len(method_data) > 0:
                fig.add_trace(
                    go.Scatter(
                        x=method_data['processing_time'],
                        y=[1 if s else 0 for s in method_data['success']],
                        mode='markers',
                        name=method,
                        marker=dict(
                            size=10,
                            color=color_map.get(method, '#6c757d'),
                            opacity=0.7,
                            line=dict(width=1, color='DarkSlateGrey')
                        ),
                        text=[f"Method: {method}<br>Success: {s}<br>Time: {t:.2f}s"
                              for s, t in zip(method_data['success'], method_data['processing_time'])],
                        hovertemplate='<b>%{text}</b><extra></extra>'
                    ),
                    row=2, col=2
                )

    # Update layout
    fig.update_layout(
        height=700,
        title_text="📊 Performance Analytics Dashboard",
        title_x=0.5,
        showlegend=True,
        template="plotly_white",
        font=dict(size=12)
    )

    # Update axes labels
    fig.update_yaxes(title_text="Success Rate (%)", row=1, col=1, range=[0, 100])
    fig.update_yaxes(title_text="Time (seconds)", row=1, col=2)
    fig.update_yaxes(title_text="Processing Time (s)", row=2, col=1)
    fig.update_yaxes(title_text="Success (1=Yes, 0=No)", row=2, col=2, tickvals=[0, 1])
    fig.update_xaxes(title_text="Method", row=1, col=1)
    fig.update_xaxes(title_text="Method", row=1, col=2)
    fig.update_xaxes(title_text="Method", row=2, col=1)
    fig.update_xaxes(title_text="Processing Time (s)", row=2, col=2)

    return fig

def create_success_timeline() -> go.Figure:
    """Create a timeline of success/failure over time"""
    if not stt_engine.performance_metrics:
        fig = go.Figure()
        fig.add_annotation(text="No timeline data available", x=0.5, y=0.5, showarrow=False)
        return fig

    df = pd.DataFrame(stt_engine.performance_metrics)
    df['timestamp'] = pd.to_datetime(df.get('timestamp', pd.Timestamp.now()))
    df = df.sort_values('timestamp')

    fig = go.Figure()

    color_map = {'Whisper': '#28a745', 'Vosk': '#007bff', 'Google API': '#dc3545'}

    for method in df['method'].unique():
        method_data = df[df['method'] == method]

        # Success points
        success_data = method_data[method_data['success'] == True]
        if len(success_data) > 0:
            fig.add_trace(go.Scatter(
                x=success_data['timestamp'],
                y=[method] * len(success_data),
                mode='markers',
                name=f'{method} - Success',
                marker=dict(color=color_map.get(method, '#28a745'), size=12, symbol='circle'),
                text=[f"Time: {t:.2f}s" for t in success_data['processing_time']]
            ))

        # Failure points
        failure_data = method_data[method_data['success'] == False]
        if len(failure_data) > 0:
            fig.add_trace(go.Scatter(
                x=failure_data['timestamp'],
                y=[method] * len(failure_data),
                mode='markers',
                name=f'{method} - Failed',
                marker=dict(color='#dc3545', size=12, symbol='x'),
                text=[f"Error: {t}" for t in failure_data['text']]
            ))

    fig.update_layout(
        title="📈 Recognition Success Timeline",
        xaxis_title="Time",
        yaxis_title="Method",
        height=400,
        showlegend=True
    )

    return fig

In [12]:
def show_analytics():
    """Show performance analytics - ENHANCED VERSION"""
    dashboard = create_performance_dashboard()
    timeline = create_success_timeline()

    if not stt_engine.performance_metrics:
        analytics_text = """
## 📈 Performance Analytics

**No performance data available yet.**

### How to get started:
1. Go to **"Single Method Test"** tab and process an audio file
2. Or go to **"Method Comparison"** tab to compare all methods
3. Return here to see your analytics!

### Expected Metrics:
- ✅ Success rates for each method
- ⏱️ Processing time comparisons
- 📊 Performance trends over time
- 📈 Method reliability analysis
"""
    else:
        df = pd.DataFrame(stt_engine.performance_metrics)
        total_tests = len(df)
        success_rate = df['success'].mean() * 100

        # Method-specific statistics
        method_stats = []
        for method in df['method'].unique():
            method_data = df[df['method'] == method]
            method_success = method_data['success'].mean() * 100
            avg_time = method_data['processing_time'].mean()
            tests_count = len(method_data)
            method_stats.append({
                'method': method,
                'success_rate': method_success,
                'avg_time': avg_time,
                'tests_count': tests_count
            })

        # Sort by success rate (descending)
        method_stats.sort(key=lambda x: x['success_rate'], reverse=True)

        analytics_text = f"""
## 📈 Performance Analytics

### 📊 Overall Statistics:
- **Total Tests Conducted:** {total_tests}
- **Overall Success Rate:** {success_rate:.1f}%
- **Methods Compared:** {len(df['method'].unique())}

### 🏆 Method Performance Ranking:
"""

        for i, stats in enumerate(method_stats, 1):
            medal = "🥇" if i == 1 else "🥈" if i == 2 else "🥉" if i == 3 else "📊"
            analytics_text += f"""
{medal} **{stats['method']}:**
   - Success Rate: {stats['success_rate']:.1f}%
   - Average Time: {stats['avg_time']:.2f}s
   - Tests: {stats['tests_count']}
"""

        # Additional insights
        best_method = method_stats[0]['method'] if method_stats else "N/A"
        fastest_method = min(method_stats, key=lambda x: x['avg_time'])['method'] if method_stats else "N/A"

        analytics_text += f"""
### 💡 Key Insights:
- **Most Accurate:** {best_method}
- **Fastest:** {fastest_method}
- **Total Processing Time:** {df['processing_time'].sum():.2f}s

### 📈 Recommendations:
"""

        if success_rate < 50:
            analytics_text += "- 🔴 **Low overall accuracy** - Try using clearer audio or different methods"
        elif success_rate < 80:
            analytics_text += "- 🟡 **Moderate accuracy** - Consider using the best performing method consistently"
        else:
            analytics_text += "- 🟢 **High accuracy** - System is performing well!"

        if any(stats['success_rate'] > 90 for stats in method_stats):
            analytics_text += f"\n- ⭐ **{best_method}** is highly reliable (over 90% success rate)"

    return analytics_text, dashboard, timeline

def clear_history():
    """Clear all history and metrics"""
    stt_engine.results_history.clear()
    stt_engine.performance_metrics.clear()

    # Create empty dashboard
    empty_dashboard = create_performance_dashboard()
    empty_timeline = create_success_timeline()

    analytics_text = """
## 📈 Performance Analytics

**🗑️ History cleared!**

### Next Steps:
1. Process new audio files in the other tabs
2. Return here to see fresh analytics
3. Build up new performance data

### What you'll see:
- Success rates for each method
- Processing time comparisons
- Performance trends
- Method recommendations
"""

    return analytics_text, empty_dashboard, empty_timeline

In [13]:
# Define CSS for better styling
css = """
.gradio-container {
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
}
.header {
    text-align: center;
    padding: 20px;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    color: white;
    border-radius: 10px;
    margin-bottom: 20px;
}
.success { color: #28a745; font-weight: bold; }
.error { color: #dc3545; font-weight: bold; }
.processing { color: #ffc107; font-weight: bold; }
.analytics-panel {
    background: #f8f9fa;
    padding: 15px;
    border-radius: 10px;
    border-left: 4px solid #007bff;
}
"""

# Create the Gradio interface
with gr.Blocks(css=css, theme=gr.themes.Soft()) as demo:

    gr.HTML("""
    <div class="header">
        <h1>🎤 Speech-to-Text Accessibility System</h1>
        <p>Convert spoken commands to text using multiple recognition methods</p>
        <p><strong>Perfect for accessibility applications and device control</strong></p>
    </div>
    """)

    with gr.Tabs():
        with gr.TabItem("🔍 Single Method Test"):
            with gr.Row():
                with gr.Column():
                    gr.Markdown("### 🎵 Upload or Record Audio")
                    audio_input_single = gr.Audio(
                        label="Choose Audio File or Record Voice",
                        type="numpy",
                        sources=["upload", "microphone"],
                        waveform_options={"show_controls": True}
                    )
                    method_selector = gr.Radio(
                        choices=["Whisper", "Vosk", "Google API"],
                        label="Recognition Method",
                        value="Whisper",
                        info="Choose which method to use for speech recognition"
                    )
                    audio_type_single = gr.Textbox(
                        label="Audio Type Description",
                        placeholder="e.g., Clear male voice, Noisy background...",
                        value="Custom"
                    )
                    process_btn_single = gr.Button("🚀 Process Audio", variant="primary", size="lg")

                with gr.Column():
                    gr.Markdown("### 📝 Recognition Results")
                    output_text_single = gr.Markdown(label="Results", value="Results will appear here...")
                    status_indicator = gr.Textbox(
                        label="Status",
                        value="Waiting for input...",
                        interactive=False
                    )

            with gr.Row():
                with gr.Column():
                    waveform_single = gr.Plot(label="📊 Waveform Visualization")
                with gr.Column():
                    spectrogram_single = gr.Plot(label="🌈 Spectrogram Visualization")

        with gr.TabItem("📊 Method Comparison"):
            with gr.Row():
                with gr.Column():
                    gr.Markdown("### 🎵 Upload or Record Audio")
                    audio_input_compare = gr.Audio(
                        label="Choose Audio File or Record Voice",
                        type="numpy",
                        sources=["upload", "microphone"],
                        waveform_options={"show_controls": True}
                    )
                    audio_type_compare = gr.Dropdown(
                        choices=[
                            "Clear male voice", "Clear female voice",
                            "Fast speech", "Noisy background", "Soft voice", "Custom"
                        ],
                        label="Audio Type Category",
                        value="Custom",
                        info="Select the type of audio for testing"
                    )
                    compare_btn = gr.Button("🔬 Compare All Methods", variant="primary", size="lg")

                with gr.Column():
                    gr.Markdown("### 📊 Comparison Summary")
                    summary_output = gr.Markdown(label="Summary", value="Comparison results will appear here...")

            gr.Markdown("### 📋 Detailed Results")
            results_table = gr.Dataframe(
                label="Method Comparison Results",
                headers=["Method", "Result", "Status", "Processing Time"],
                interactive=False,
                wrap=True
            )

            comparison_plot = gr.Plot(label="📈 Method Performance Comparison")

            with gr.Row():
                with gr.Column():
                    waveform_compare = gr.Plot(label="📊 Waveform")
                with gr.Column():
                    spectrogram_compare = gr.Plot(label="🌈 Spectrogram")

        with gr.TabItem("📈 Analytics Dashboard"):
            with gr.Row():
                with gr.Column():
                    gr.Markdown("""
                    <div class="analytics-panel">
                    <h3>📊 Performance Analytics</h3>
                    <p>View comprehensive analytics and performance metrics for all speech recognition methods.</p>
                    </div>
                    """)
                    with gr.Row():
                        analytics_btn = gr.Button("🔄 Refresh Analytics", variant="primary", size="lg")
                        clear_btn = gr.Button("🗑️ Clear History", variant="secondary", size="lg")

                    gr.Markdown("""
                    **📈 What you'll see:**
                    - Success rates for each method
                    - Processing time analysis
                    - Performance trends
                    - Method recommendations

                    **💡 Tips:**
                    - Process multiple audio files for better insights
                    - Compare different audio types
                    - Use the best performing method for your needs
                    """)

                with gr.Column():
                    analytics_text = gr.Markdown(
                        label="Analytics Summary",
                        value="## 📈 Performance Analytics\n\nProcess some audio files to see analytics here!"
                    )

            gr.Markdown("### 📊 Performance Dashboard")
            analytics_dashboard = gr.Plot(label="Main Dashboard")

            gr.Markdown("### 📈 Success Timeline")
            analytics_timeline = gr.Plot(label="Timeline")

        with gr.TabItem("ℹ️ About & Help"):
            gr.Markdown("""
            ## 🎯 About This System

            This Speech-to-Text Accessibility System provides multiple methods for converting spoken audio to text,
            specifically designed for accessibility applications.

            ### 🔧 Available Methods:

            | Method | Type | Best For | Pros | Cons |
            |--------|------|----------|------|------|
            | **Whisper** | Offline | General purpose | High accuracy, multiple languages | Slower processing |
            | **Vosk** | Offline | Real-time apps | Fast, lightweight | Lower accuracy |
            | **Google API** | Online | High accuracy | Very accurate, robust | Requires internet |

            ### 📊 Analytics Features:
            - Real-time performance tracking
            - Success rate comparisons
            - Processing time analysis
            - Method recommendations
            - Historical data visualization
            """)

    # Event handlers
    process_btn_single.click(
        fn=process_single_method,
        inputs=[audio_input_single, method_selector, audio_type_single],
        outputs=[output_text_single, waveform_single, spectrogram_single, status_indicator]
    )

    compare_btn.click(
        fn=process_comparison,
        inputs=[audio_input_compare, audio_type_compare],
        outputs=[summary_output, results_table, comparison_plot, waveform_compare, spectrogram_compare]
    )

    # Updated analytics handlers to include timeline
    analytics_btn.click(
        fn=show_analytics,
        inputs=[],
        outputs=[analytics_text, analytics_dashboard, analytics_timeline]
    )

    clear_btn.click(
        fn=clear_history,
        inputs=[],
        outputs=[analytics_text, analytics_dashboard, analytics_timeline]
    )

    # Initial load
    demo.load(
        fn=show_analytics,
        inputs=[],
        outputs=[analytics_text, analytics_dashboard, analytics_timeline]
    )

# Launch the interface
print("🚀 Launching Gradio Interface...")
print("📊 Performance dashboard is now fixed!")
print("⏳ This may take a few moments to load...")

try:
    demo.launch(share=True, debug=False)
except Exception as e:
    print(f"Note: Could not create public link. Launching locally...")
    demo.launch(debug=False)

/usr/local/lib/python3.12/dist-packages/gradio/components/audio.py:200: UserWarning:

The `show_controls` parameter is deprecated and will be removed in a future release. Use `show_recording_waveform` instead.



🚀 Launching Gradio Interface...
📊 Performance dashboard is now fixed!
⏳ This may take a few moments to load...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ad5969398b714d5a60.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
